In [ ]:
import os
os.environ["http_proxy"] = "http://proxy.cmu.edu:3128"
os.environ["https_proxy"] = "http://proxy.cmu.edu:3128"
from torchmetrics.image.fid import FrechetInceptionDistance
from train import test_acc, domain_test_numpy, source_domain_numpy
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm import tqdm



In [ ]:
device = "cuda:0"


config = {'base_root':'../data', 'source_dataset':'mnist,svhn,syn_digits', 'target_dataset':'mnist_m', 'batch_size':256,\
             'resolution':28,'num_workers':4, 'seed':1, 'num_epochs': 100}

transforms_test = transforms.Compose([transforms.Resize(28),transforms.ToTensor(),transforms.Normalize([0.5], [0.5])])
real_set = source_domain_numpy(root= config['base_root'], root_list='mnist_m', transform=transforms_test)
real_loader = DataLoader(real_set, batch_size=config['batch_size'], shuffle=True, num_workers=5)

In [ ]:
fake_set = source_domain_numpy(root= config['base_root'], root_list='mnist_m', transform=transforms_test)
fake_loader = DataLoader(real_set, batch_size=config['batch_size'], shuffle=True, num_workers=5)


In [ ]:
fid = FrechetInceptionDistance(normalize=True).to(device)


for data, data2 in tqdm(zip(real_loader, fake_loader)):
    img, labels, domain = data
    img2, labels, domain = data2
    fid.update(img.to(device), real=True)
    fid.update(img2.to(device), real=False)

#for data in tqdm(real_loader):
#    img, labels, domain = data
#    fid.update(img.to(device), real=False)

print(f"FID: {float(fid.compute())}")